### Import Libs

In [1]:
# import libs
import numpy as np
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
import matplotlib.pyplot as plt
import pandas as pd 
from pyproj import CRS
import constant as c
from shapely.wkt import loads
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings; warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor  # or RandomForestClassifier
# kernel must be msd2
import pandas as pd
import geopandas as gpd
from shapely.geometry import *
import sys
from shapely import wkt
sys.path.append('../')
from util.epsg_transform import *
from util.distance import *
import constant as c
import os
import matplotlib.pyplot as plt

In [2]:
# # then read the data
# df_2020 = pd.read_csv('../asset/preprocess/df_2020.csv', index_col=0)
# df_2020['geometry_grids'] = df_2020['geometry_grids'].apply(lambda x: loads(x))
# df_2021 = pd.read_csv('../asset/preprocess/df_2021.csv', index_col=0)
# df_2021['geometry_grids'] = df_2021['geometry_grids'].apply(lambda x: loads(x))
# df_2022 = pd.read_csv('../asset/preprocess/df_2022.csv', index_col=0)
# df_2022['geometry_grids'] = df_2022['geometry_grids'].apply(lambda x: loads(x))

# crs = 'EPSG:5179'  # Specify the coordinate reference system
# gdf_2020 = gpd.GeoDataFrame(df_2020, geometry=df_2020['geometry_grids'], crs=crs)
# gdf_2021 = gpd.GeoDataFrame(df_2021, geometry=df_2021['geometry_grids'], crs=crs)
# gdf_2022 = gpd.GeoDataFrame(df_2022, geometry=df_2022['geometry_grids'], crs=crs)

# gdf_2020_scbd = gdf_2020[gdf_2020['sigungunm'].isin(c.SCBD_NMS)]
# gdf_2020_gbd = gdf_2020[gdf_2020['sigungunm'].isin(c.GBD_NMS)]
# gdf_2020_ybd = gdf_2020[gdf_2020['sigungunm'].isin(c.YBD_NMS)]
# gdf_2021_scbd = gdf_2021[gdf_2021['sigungunm'].isin(c.SCBD_NMS)]
# gdf_2021_gbd = gdf_2021[gdf_2021['sigungunm'].isin(c.GBD_NMS)]
# gdf_2021_ybd = gdf_2021[gdf_2021['sigungunm'].isin(c.YBD_NMS)]
# gdf_2022_scbd = gdf_2022[gdf_2022['sigungunm'].isin(c.SCBD_NMS)]
# gdf_2022_gbd = gdf_2022[gdf_2022['sigungunm'].isin(c.GBD_NMS)]
# gdf_2022_ybd = gdf_2022[gdf_2022['sigungunm'].isin(c.YBD_NMS)]

In [3]:
df_grids = pd.read_csv('../asset/preprocess/grid/df_grid_pop_od_drr_ebit.csv', index_col = 0) # grid 기준: 각 grid에 pop, od 데이터가 옆으로 추가된 형태
gdf_cbds = pd.read_csv("../asset/preprocess/grid/gdf_cbds.csv", index_col = 0) # building 기준: 각 building에 feature 데이터가 옆으로 추가된 형태

### Get preprocess bd tval data

In [15]:
df_tval = pd.read_csv("../asset/verification/df_bd_tval.csv", index_col=0)
df_tval['pnu'] = df_tval['pnu'].astype(str)
df_tval = df_tval[df_tval['pnu'].str[:5].isin(c.CBD_CDS)]
df_tval = df_tval[['tyyyymm', 'totalprc', 'pnu', 'x', 'y']] # pnu, 거래가, 거래일
df_tval['tyear'] = df_tval['tyyyymm'] // 100
x_5179, y_5179 = coord_tranformation(list(df_tval.x), list(df_tval.y), "epsg:4326", "epsg:5179")
df_tval['x_5179'] = x_5179
df_tval['y_5179'] = y_5179
df_tval['geometry'] = df_tval.apply(lambda row: Point([row['x_5179'], row['y_5179']]), axis = 1)
df_tval = gpd.GeoDataFrame(df_tval, geometry='geometry', crs = 'EPSG:5179')
df_tval['totalprc'] = df_tval['totalprc'] / 1e8 
df_tval.head(2)

,tyyyymm,totalprc,pnu,x,y,tyear,x_5179,y_5179,geometry
0,202112,1.127068,1114011800103950000,126.975447,37.555493,2021,953668.907172,1.950812e+06,POINT (953668.907 1950812.131)
1,202007,0.939334,1114011800108310000,126.974415,37.557838,2020,953579.196053,1.951073e+06,POINT (953579.196 1951072.894)


In [16]:
# Get df_grid
df_grid = pd.read_csv('../asset/preprocess/grid/df_grid.csv', index_col = 0)
df_grid = df_grid.rename(columns = {'geometry': 'geometry_grids'})
df_grid['geometry_grids'] = df_grid['geometry_grids'].apply(wkt.loads)
df_grid = df_grid[['idx', 'ADM_SECT_C', 'geometry_grids']]
df_grid = df_grid.rename(columns={'idx': 'grid_idx'})
df_grid = gpd.GeoDataFrame(df_grid, crs = 'EPSG:5179', geometry = 'geometry_grids')
df_grid = df_grid.to_crs({'init':'epsg:5179'})
df_grid['grid_idx'] = df_grid['grid_idx'].astype(str)
df_tval_grid = gpd.sjoin(df_tval, df_grid, op = 'within', how = 'left')
df_tval_grid = df_tval_grid.dropna(subset=['grid_idx'])
df_tval_grid['grid_idx'] = df_tval_grid['grid_idx'].astype(int)

### Experiment Settings

In [17]:
# experiment settings
CBD_NM = "gbd"

In [18]:
crs = 'EPSG:5179'  # Specify the coordinate reference system
tval_cbd = pd.DataFrame()
for YEAR in ['2020', '2021', '2022']:
    fpath = f"../asset/experiment/cbdindex/cbdi_{CBD_NM}_{YEAR}.csv"
    cbdi_df = pd.read_csv(fpath)
    cbdi_df['geometry_grids'] = cbdi_df['geometry_grids'].apply(lambda x: loads(x))
    cbdi_df = gpd.GeoDataFrame(cbdi_df, geometry=cbdi_df['geometry_grids'], crs=crs)
    tval = df_tval_grid[df_tval_grid['tyear'] == int(YEAR)]
    concat_df = pd.merge(tval[['grid_idx', 'totalprc']], cbdi_df, on='grid_idx', suffixes=('', ''), how='left')
    concat_df = concat_df.dropna(subset=['cbdi'])
    tval_cbd = pd.concat([tval_cbd, concat_df], ignore_index = True)
tval_cbd = gpd.GeoDataFrame(tval_cbd, geometry=cbdi_df['geometry_grids'], crs=crs)

In [19]:
# Setting COLS
SIM_CAL_COLS = c.SIM_CAL_COLS
STANDARDIZE_COLS = c.STANDARDIZE_COLS
PP_COLS = c.PP_COLS
BS_COLS = c.BS_COLS
BD_COLS = c.BD_COLS

VAR_COLS = [
    'pp_pop', # X1
    'pp_od', # X2
    'pp_drr', # X3
    'bs_ebit', # X4
    'bs_gas', # X5
    'bs_elct', # X6
    'bd_platarea', # X7
    'bd_archarea', # X8
    'bd_totarea', # X9
    'bd_totflrcnt', # X10
    'bd_elvtent', # X11
    'bd_height', # X12
    'bd_vintage', # X13
    'bd_ilp'
    ]

TARGET_COL = ['totalprc']

In [20]:
# Filling any data point that is nan -> fill by mean value of column

X = tval_cbd[VAR_COLS].values
y = tval_cbd[TARGET_COL].values.reshape((-1,1)) 
print(X.shape)
print(y.shape)

lat = tval_cbd.centroid.x
lng = tval_cbd.centroid.y
coords = list(zip(lat,lng))

# calibrate GWR MODEL
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search(bw_min=2)
gwr_results = GWR(coords, y, X, gwr_bw).fit()

# # append to df
# cbdi_df['gwr_const'] = gwr_results.params[:,0]
# for i, col_nm in enumerate(cbdi_df):
#     cbdi_df[f'gwr_{col_nm}'] = gwr_results.params[:,i+1]
# cbdi_df['gwr_r2'] = gwr_results.localR2

print(f'{YEAR}_{CBD_NM}')
# https://mgwr.readthedocs.io/en/latest/_modules/mgwr/gwr.html#GWRResults.summary
gwr_results.summary()

(75, 14)
(75, 1)
2022_gbd
Model type                                                         Gaussian
Number of observations:                                                  75
Number of covariates:                                                    15

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                              0.062
Log-likelihood:                                                     159.784
AIC:                                                               -289.568
AICc:                                                              -278.188
BIC:                                                               -258.987
R2:                                                                   0.667
Adj. R2:                                                              0.590

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- --

In [21]:
# Filling any data point that is nan -> fill by mean value of column

X = tval_cbd[VAR_COLS+['cbdi']].values
y = tval_cbd[TARGET_COL].values.reshape((-1,1)) 
print(X.shape)
print(y.shape)

lat = tval_cbd.centroid.x
lng = tval_cbd.centroid.y
coords = list(zip(lat,lng))

# calibrate GWR MODEL
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search(bw_min=2)
gwr_results = GWR(coords, y, X, gwr_bw).fit()

# # append to df
# cbdi_df['gwr_const'] = gwr_results.params[:,0]
# for i, col_nm in enumerate(cbdi_df):
#     cbdi_df[f'gwr_{col_nm}'] = gwr_results.params[:,i+1]
# cbdi_df['gwr_r2'] = gwr_results.localR2

print(f'{YEAR}_{CBD_NM}')
# https://mgwr.readthedocs.io/en/latest/_modules/mgwr/gwr.html#GWRResults.summary
gwr_results.summary()

(75, 15)
(75, 1)
2022_gbd
Model type                                                         Gaussian
Number of observations:                                                  75
Number of covariates:                                                    16

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                              0.057
Log-likelihood:                                                     163.060
AIC:                                                               -294.120
AICc:                                                              -281.383
BIC:                                                               -254.675
R2:                                                                   0.695
Adj. R2:                                                              0.618

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- --